## Data Cleaning

Download dataset from:
https://drive.google.com/drive/folders/1pYdL0E0OHyUkHl5LZZyQMTfktXareqAZ?usp=sharing


There are multiple datasets that are required to run the various strategies that I am proposing, and some of the dataset are not cleaned. Thus, there is a need to clean them. <br /> 

<!-- The Datasets in question are:
- US equities with market cap of < 300 million at the start of the time period that was selected
- Fundamentals of US equities based on the tickers that were filtered

***need to add more***
 -->

The dataset in use here is from the Sharadar Core US Equities Bundle which includes 5 other sharadar products:
[Sharadar Core US Equities Bundle](https://www.quandl.com/databases/SFA/documentation)
1. Core US Fundamentals Data
2. Core US Insiders Data
3. Core US Institutional Investors Data
4. Sharadar Equity Prices
5. Sharadar Fund Prices

According to Quandl, the provider of the dataset, by default data are presented after adjustment for stock splits and stock dividends, but not for cash dividends.





## Target Markets for PEAD strategy
- Exploration and formulation of the PEAD strategy particularly in more illiquid markets such as the US microcap stock market.
- filtering for stocks with < 300 million USD market cap at the start of the period


## Factor Exploration for PEAD strategy
- there are several factors that could be taken into consideration with PEAD. the traditional PEAD strategy only be looking into the effects that earnings releases have on the stock price.
- However, we are also looking into variations of PEAD which includes the effects that factors such as valuation, coporate actions, and others have to stock price.

### Momentum Strategy Mixed with Company Fundamentals ( incomplete)
- For momentum strategy related to earnings, I can add fundamental stuff that I’ve observed too.
- Like earnings beat when valuation is dirt cheap. Earnings beat by more than 20% would be very positive for example, and also if valuation is EV/ebitda low

Operating leverage too
- Type as well, for example if fda approval won


### Technical Stategy


- Maybe I can addin factors like mutual fund forced sale index inclusion, short covering and stuff 



In [1]:
## remember to export all the data here as CSV. in case u ever lose it once the subscription expires

In [1]:
#usual import from workflow
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LassoCV, LinearRegression, RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from datetime import datetime


## Filter by Market Capitalisation
- we start off the data cleaning process by filtering out the market cap that we are going to look at

### Data Description
- Read in the table csv which consists of all the fundamentals data regarding the stocks with tickers in alphabetical order (A to ZYXI)
- the calendar dates are between 2015-03-31 and 2020-09-30, however, it is not sorted
- First, we have to filter only the stocks that have a market capitalisation of 300 mil or less at the starting date, 2015-03-31, (They are sometimes known as microcap stocks). 
- Traditionally, instituionals are less likely to own these stocks, thus it is assumed that such stocks will be less efficient (rationale will be explained in the README)

source: https://www.investopedia.com/terms/p/pennystock.asp

In [2]:
#read the table csv data
#this gives us the quarterly data of the SF1 file, it does not mean that our starting date is 2015-03-31
table = pd.read_csv('../dataset/SF1.csv')


In [3]:
# we need the set the datekey as the index for our time series data because it is the SEC filing date
table['datekey'] = table['datekey'].astype('datetime64[ns]')
table.set_index('datekey', inplace = True)



In [4]:
table.sort_index()
# we sort it based on the index, which is the datekey

,ticker,dimension,calendardate,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,assetsnc,...,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital
datekey,,,,,,,,,,,,,,,,,,,,,
2015-01-30,SAIC,MRQ,2015-03-31,2015-01-30,2020-12-04,-5000000.0,1.389000e+09,NaN,9.420000e+08,4.470000e+08,...,45687892.0,45687892.0,NaN,20.837,1.008000e+09,0.0,19000000.0,1.900000e+07,22.063,3.900000e+08
2015-01-30,SAIC,MRY,2015-12-31,2015-01-30,2020-12-04,-5000000.0,1.389000e+09,1.407000e+09,9.420000e+08,4.470000e+08,...,45687892.0,45687892.0,NaN,85.033,1.008000e+09,0.0,82000000.0,1.900000e+07,22.063,3.900000e+08
2015-01-30,SAIC,MRT,2015-03-31,2015-01-30,2020-12-04,-5000000.0,1.389000e+09,1.407000e+09,9.420000e+08,4.470000e+08,...,45687892.0,45687892.0,NaN,85.033,1.008000e+09,0.0,82000000.0,1.900000e+07,22.063,3.900000e+08
2015-01-31,KR,MRY,2015-12-31,2015-01-31,2020-12-15,-812000000.0,3.049700e+10,2.967775e+10,8.911000e+09,2.158600e+10,...,982648792.0,981000000.0,993000000.0,110.566,2.743600e+10,0.0,902000000.0,1.496000e+09,27.967,-2.481000e+09
2015-01-31,KR,MRT,2015-03-31,2015-01-31,2020-12-15,-812000000.0,3.049700e+10,2.967775e+10,8.911000e+09,2.158600e+10,...,982648792.0,981000000.0,993000000.0,110.566,2.743600e+10,0.0,902000000.0,1.496000e+09,27.967,-2.481000e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-18,CEI,ART,2020-09-30,2020-09-30,2020-12-18,0.0,1.179545e+07,1.012546e+07,1.479009e+06,1.031644e+07,...,25000000.0,19815872.0,19815872.0,0.014,1.179545e+07,0.0,0.0,3.000000e+03,0.595,-1.132760e+05
2020-12-18,CEI,ARQ,2020-09-30,2020-09-30,2020-12-18,0.0,1.179545e+07,NaN,1.479009e+06,1.031644e+07,...,25000000.0,19815872.0,19815872.0,0.003,1.179545e+07,0.0,0.0,3.000000e+03,0.595,-1.132760e+05
2020-12-18,AVGO,ARY,2020-12-31,2020-11-01,2020-12-18,-108000000.0,7.593300e+10,7.944625e+10,1.189500e+10,6.403800e+10,...,406713118.0,402000000.0,421000000.0,59.423,1.570400e+10,0.0,-518000000.0,0.000000e+00,39.065,5.524000e+09


In [5]:
pd.set_option('display.max_columns', None)
table.head(10)
#show the first 10 rows


,ticker,dimension,calendardate,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,assetsnc,assetturnover,bvps,capex,cashneq,cashnequsd,cor,consolinc,currentratio,de,debt,debtc,debtnc,debtusd,deferredrev,depamor,deposits,divyield,dps,ebit,ebitda,ebitdamargin,ebitdausd,ebitusd,ebt,eps,epsdil,epsusd,equity,equityavg,equityusd,ev,evebit,evebitda,fcf,fcfps,fxusd,gp,grossmargin,intangibles,intexp,invcap,invcapavg,inventory,investments,investmentsc,investmentsnc,liabilities,liabilitiesc,liabilitiesnc,marketcap,ncf,ncfbus,ncfcommon,ncfdebt,ncfdiv,ncff,ncfi,ncfinv,ncfo,ncfx,netinc,netinccmn,netinccmnusd,netincdis,netincnci,netmargin,opex,opinc,payables,payoutratio,pb,pe,pe1,ppnenet,prefdivis,price,ps,ps1,receivables,retearn,revenue,revenueusd,rnd,roa,roe,roic,ros,sbcomp,sgna,sharefactor,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital
datekey,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-31,ZYXI,MRY,2019-12-31,2019-12-31,2020-10-27,0.0,28277000.0,23426250.0,22566000.0,5711000.0,1.941,0.609,-160000.0,14369000.0,14369000.0,8814000.0,9492000.0,4.342,0.437,4683000.0,1256000.0,3427000.0,4683000.0,0.0,778000.0,0.0,0.000,0.00,11946000.0,12724000.0,0.280,12724000.0,11946000.0,11941000.0,0.29,0.28,0.29,19742000.0,15521500.0,19742000.0,250341378.0,21.0,19.675,6143000.0,0.189,1.0,36658000.0,0.806,0.0,5000.0,13394000.0,11741250.0,2378000.0,0.0,0.0,0.0,8624000.0,5197000.0,3427000.0,257689378.0,3912000.0,0.0,50000.0,-19000.0,-2262000.0,-2231000.0,-160000.0,0.0,6303000.0,0.0,9492000.0,9492000.0,9492000.0,0.0,0.0,0.209,25592000.0,11066000.0,2141000.0,0.000,13.053,27.148,27.138,4869000.0,0.0,7.87,5.667,5.614,5833000.0,14356000.0,45472000.0,45472000.0,0.0,0.405,0.612,1.017,0.263,820000.0,25592000.0,1.0,32743250.0,32439000.0,33963000.0,1.402,28277000.0,513000.0,2449000.0,52000.0,0.872,17369000.0
2018-12-31,ZYXI,MRY,2018-12-31,2018-12-31,2020-10-27,0.0,19251000.0,13312500.0,14324000.0,4927000.0,2.398,0.289,-1082000.0,10442000.0,10442000.0,6038000.0,9552000.0,2.051,1.062,3662000.0,685000.0,2977000.0,3662000.0,0.0,448000.0,0.0,0.024,0.07,10370000.0,10818000.0,0.339,10818000.0,10370000.0,10216000.0,0.29,0.28,0.29,9380000.0,7889250.0,9380000.0,86517245.0,8.0,7.998,8326000.0,0.256,1.0,25879000.0,0.811,0.0,154000.0,5488000.0,2334250.0,837000.0,0.0,0.0,0.0,9960000.0,6983000.0,2977000.0,94893245.0,4563000.0,0.0,-3256000.0,-507000.0,0.0,-3763000.0,-1082000.0,0.0,9408000.0,0.0,9552000.0,9552000.0,9552000.0,0.0,0.0,0.299,15509000.0,10370000.0,1552000.0,0.241,10.117,9.934,10.138,3888000.0,0.0,2.94,2.973,2.994,2791000.0,4864000.0,31917000.0,31917000.0,0.0,0.718,1.211,4.443,0.325,370000.0,15509000.0,1.0,32276614.0,32503000.0,34043000.0,0.982,19251000.0,725000.0,664000.0,688000.0,0.592,7341000.0
2017-12-31,ZYXI,MRY,2017-12-31,2017-12-31,2020-10-27,0.0,8929000.0,5669750.0,8371000.0,558000.0,4.133,0.155,-87000.0,5935000.0,5935000.0,4819000.0,7365000.0,2.079,0.807,354000.0,354000.0,0.0,354000.0,0.0,286000.0,0.0,0.000,0.00,8944000.0,9230000.0,0.394,9230000.0,8944000.0,7494000.0,0.23,0.22,0.23,4991000.0,496250.0,4991000.0,102290369.0,11.0,11.082,8173000.0,0.254,1.0,18613000.0,0.794,0.0,1450000.0,-679000.0,-562750.0,423000.0,0.0,0.0,0.0,4027000.0,4027000.0,0.0,104405369.0,5318000.0,0.0,-183000.0,-2517000.0,0.0,-2855000.0,-87000.0,0.0,8260000.0,0.0,7365000.0,7365000.0,7365000.0,0.0,0.0,0.314,9669000.0,8944000.0,2255000.0,0.000,20.919,14.176,13.826,188000.0,0.0,3.18,4.456,4.364,2185000.0,-2411000.0,23432000.0,23432000.0,0.0,1.299,14.841,-15.893,0.382,294000.0,9669000.0,1.0,32831877.0,32156000.0,33196000.0,0.729,8929000.0,0.0,129000.0,0.0,0.278,4344000.0
2016-12-31,ZYXI,MRY,2016-12-31,2016-12-31,2020-10-27,0.0,4091000.0,4343250.0,3422000.0,669000.0,3.065,-0.119,-226000.0,302000.0,302000.0,3517000.0,69000.0,0.442,-2.126,3025000.0,2889000.0,136000.0,3025000.0,0.0,435000.0,0.0,0.000,0.00,436000.0,871000.0,0.065,871000.0,436000.0,84000.0,0.00,0.00,0.00,-3713

In [6]:
pd.set_option('display.max_columns', None)
table.tail()
#show the last 5 rows

,ticker,dimension,calendardate,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,assetsnc,assetturnover,bvps,capex,cashneq,cashnequsd,cor,consolinc,currentratio,de,debt,debtc,debtnc,debtusd,deferredrev,depamor,deposits,divyield,dps,ebit,ebitda,ebitdamargin,ebitdausd,ebitusd,ebt,eps,epsdil,epsusd,equity,equityavg,equityusd,ev,evebit,evebitda,fcf,fcfps,fxusd,gp,grossmargin,intangibles,intexp,invcap,invcapavg,inventory,investments,investmentsc,investmentsnc,liabilities,liabilitiesc,liabilitiesnc,marketcap,ncf,ncfbus,ncfcommon,ncfdebt,ncfdiv,ncff,ncfi,ncfinv,ncfo,ncfx,netinc,netinccmn,netinccmnusd,netincdis,netincnci,netmargin,opex,opinc,payables,payoutratio,pb,pe,pe1,ppnenet,prefdivis,price,ps,ps1,receivables,retearn,revenue,revenueusd,rnd,roa,roe,roic,ros,sbcomp,sgna,sharefactor,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital
datekey,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-06-07,A,ARQ,2016-03-31,2016-04-30,2020-12-18,-299000000.0,7.640000e+09,NaN,3.488000e+09,4.152000e+09,NaN,12.767,-25000000.0,2.139000e+09,2.139000e+09,489000000.0,91000000.0,3.079,0.835,1.889000e+09,235000000.0,1.654000e+09,1.889000e+09,279000000.0,64000000.0,0.0,0.009,0.115,135000000.0,199000000.0,0.195,199000000.0,135000000.0,117000000.0,0.28,0.28,0.28,4.162000e+09,NaN,4.162000e+09,1.464262e+10,25.0,17.247,231000000.0,0.709,1.0,530000000.0,0.520,3.046000e+09,18000000.0,3.211000e+09,NaN,555000000.0,157000000.0,0.0,157000000.0,3.475000e+09,1.133000e+09,2.342000e+09,1.489262e+10,0.000000e+00,0.0,-86000000.0,155000000.0,-37000000.0,30000000.0,-108000000.0,-83000000.0,256000000.0,30000000.0,91000000.0,91000000.0,91000000.0,0.0,0.0,0.089,399000000.0,131000000.0,220000000.0,0.411,3.578,32.375,32.914,610000000.0,0.0,45.75,3.636,3.641,602000000.0,5.720000e+09,1.019000e+09,1.019000e+09,81000000.0,NaN,NaN,NaN,NaN,15000000.0,318000000.0,1.0,325521700.0,326000000.0,328000000.0,3.126,4.594000e+09,0.0,26000000.0,0.0,14.092,2.355000e+09
2016-03-08,A,ARQ,2015-12-31,2016-01-31,2020-12-18,-438000000.0,7.302000e+09,NaN,3.399000e+09,3.903000e+09,NaN,12.295,-38000000.0,1.931000e+09,1.931000e+09,491000000.0,123000000.0,3.589,0.804,1.733000e+09,80000000.0,1.653000e+09,1.733000e+09,277000000.0,66000000.0,0.0,0.011,0.115,160000000.0,226000000.0,0.220,226000000.0,160000000.0,142000000.0,0.37,0.37,0.37,4.045000e+09,NaN,4.045000e+09,1.228676e+10,22.0,14.966,66000000.0,0.201,1.0,537000000.0,0.522,2.991000e+09,18000000.0,3.166000e+09,NaN,554000000.0,76000000.0,0.0,76000000.0,3.254000e+09,9.470000e+08,2.307000e+09,1.248476e+10,2.450000e+08,-235000000.0,-176000000.0,80000000.0,-38000000.0,-132000000.0,-28000000.0,0.0,104000000.0,-16000000.0,123000000.0,123000000.0,123000000.0,0.0,0.0,0.120,382000000.0,155000000.0,250000000.0,0.311,3.086,27.621,28.007,594000000.0,0.0,38.09,3.090,3.102,617000000.0,5.666000e+09,1.028000e+09,1.028000e+09,78000000.0,NaN,NaN,NaN,NaN,21000000.0,304000000.0,1.0,327770081.0,329000000.0,332000000.0,3.125,4.311000e+09,0.0,19000000.0,0.0,13.103,2.452000e+09
2015-12-21,A,ARQ,2015-09-30,2015-10-31,2020-12-18,-391000000.0,7.479000e+09,NaN,3.686000e+09,3.793000e+09,NaN,12.627,-26000000.0,2.245000e+09,2.245000e+09,500000000.0,143000000.0,3.777,0.794,1.655000e+09,0.0,1.655000e+09,1.655000e+09,258000000.0,60000000.0,0.0,0.010,0.100,159000000.0,219000000.0,0.212,219000000.0,159000000.0,143000000.0,0.43,0.43,0.43,4.167000e+09,NaN,4.167000e+09,1.297254e+10,25.0,17.024,211000000.0,0.639,1.0,535000000.0,0.517,2.811000e+09,16000000.0,3.102000e+09,NaN,541000000.0,86000000.0,0.0,86000000.0,3.309000e+09,9.760000e+08,2.333000e+09,1.356254e+10,-1.026000e+09,-8000000.0,1000000.0,0.0,-33000000.0,-29000000.0,-275000000.0,-240000000.0,237000000.0,-5000000.0,143000000.0,143000000.0,143000000.0,0.0,0.0,0.138,379000000.0,156000000.0,279000000.0,0.233,3.255,33.822,34.025,604000000.0,0.0,40.83,3.359,3.367,606000000.0,5.581000e+09,1.035000e+09,1.035000e+09,82000000.0,

In [7]:
table.dtypes
#check datatypes of each column

ticker             object
dimension          object
calendardate       object
reportperiod       object
lastupdated        object
                   ...   
taxassets         float64
taxexp            float64
taxliabilities    float64
tbvps             float64
workingcapital    float64
Length: 110, dtype: object

In [8]:
fundamentals_filtered_1st_filing = table.loc[(table['calendardate'] <= '2015-03-31') & (table['dimension'] == 'ARQ') & (table['marketcap'] <= 300000000) ]
#filter the table by calendar date, dimension of ART means as reported, and market cap of less than 300 mil
#for the date, we want to find those that are less than 300 mil at the 1st quarter filing date of our data set which is 2015-03-31
#this does not mean that our starting data is 2015-03-31
#this is the fundamentals specifically for the day 2015-03-31, so there is no point in exporting it


In [9]:
fundamentals_filtered_1st_filing
#this dataframe has the tickers that we are concerned with and also the fundamentals that we want

,ticker,dimension,calendardate,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,assetsnc,assetturnover,bvps,capex,cashneq,cashnequsd,cor,consolinc,currentratio,de,debt,debtc,debtnc,debtusd,deferredrev,depamor,deposits,divyield,dps,ebit,ebitda,ebitdamargin,ebitdausd,ebitusd,ebt,eps,epsdil,epsusd,equity,equityavg,equityusd,ev,evebit,evebitda,fcf,fcfps,fxusd,gp,grossmargin,intangibles,intexp,invcap,invcapavg,inventory,investments,investmentsc,investmentsnc,liabilities,liabilitiesc,liabilitiesnc,marketcap,ncf,ncfbus,ncfcommon,ncfdebt,ncfdiv,ncff,ncfi,ncfinv,ncfo,ncfx,netinc,netinccmn,netinccmnusd,netincdis,netincnci,netmargin,opex,opinc,payables,payoutratio,pb,pe,pe1,ppnenet,prefdivis,price,ps,ps1,receivables,retearn,revenue,revenueusd,rnd,roa,roe,roic,ros,sbcomp,sgna,sharefactor,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital
datekey,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-05-12,ZYXI,ARQ,2015-03-31,2015-03-31,2020-10-27,0.0,6.416000e+06,NaN,5103000.0,1313000.0,NaN,-0.067,0.0,406000.0,406000.0,1245000.0,-904000.0,0.618,-4.117,4913000.0,4615000.0,298000.0,4913000.0,14000.0,0.0,0.0,0.714,0.00,-764000.0,-764000.0,-0.240,-764000.0,-764000.0,-896000.0,-0.03,-0.03,-0.03,-2082000.0,NaN,-2082000.0,11073959.0,-2.0,-2.356,163000.0,0.005,1.0,1938000.0,0.609,115000.0,132000.0,2.547000e+06,NaN,1640000.0,0.0,0.0,0.0,8.572000e+06,8261000.0,311000.0,6566959.0,285000.0,0.0,0.0,82000.0,0.0,82000.0,40000.0,0.0,163000.0,0.0,-896000.0,-896000.0,-896000.0,0.0,-8000.0,-0.281,2710000.0,-772000.0,2700000.0,0.000,-3.154,-1.159,-1.167,1140000.0,0.0,0.210,0.590,0.590,2924000.0,-7830000.0,3183000.0,3183000.0,0.0,NaN,NaN,NaN,NaN,0.0,2710000.0,1.0,31271234.0,31271234.0,31271234.0,0.102,6301000.0,0.0,0.0,0.0,0.201,-3158000.0
2015-05-12,ZSAN,ARQ,2015-03-31,2015-03-31,2020-11-13,0.0,6.642900e+07,NaN,56792000.0,9637000.0,NaN,101.222,-59000.0,55952000.0,55952000.0,0.0,-4583000.0,15.568,0.341,14680000.0,1457000.0,13223000.0,14680000.0,68000.0,633000.0,0.0,0.000,0.00,-4091000.0,-3458000.0,-15.862,-3458000.0,-4091000.0,-4583000.0,-9.40,-9.40,-9.40,49528000.0,NaN,49528000.0,72167833.0,NaN,NaN,-5486000.0,-11.212,1.0,218000.0,1.000,0.0,492000.0,2.150900e+07,NaN,0.0,0.0,0.0,0.0,1.690100e+07,3648000.0,13253000.0,113439833.0,54703000.0,0.0,61617000.0,-348000.0,0.0,60197000.0,-67000.0,-8000.0,-5427000.0,0.0,-4583000.0,-4583000.0,-4583000.0,0.0,0.0,-21.023,4369000.0,-4151000.0,1208000.0,0.000,2.290,NaN,NaN,9106000.0,0.0,190.000,NaN,NaN,142000.0,-143047000.0,218000.0,218000.0,3070000.0,NaN,NaN,NaN,NaN,29000.0,1299000.0,1.0,597052.0,489300.0,489300.0,0.446,66429000.0,0.0,0.0,0.0,135.763,53144000.0
2015-06-09,ZQKSQ,ARQ,2015-03-31,2015-04-30,2019-04-25,34517000.0,1.138592e+09,NaN,674279000.0,464313000.0,NaN,-0.085,-7025000.0,55395000.0,55395000.0,176254000.0,-37594000.0,2.176,-78.810,820767000.0,35285000.0,785482000.0,820767000.0,0.0,9707000.0,0.0,0.000,0.00,-15826000.0,-6119000.0,-0.018,-6119000.0,-15826000.0,-33866000.0,-0.22,-0.22,-0.22,-14633000.0,NaN,-14633000.0,911069364.0,-4.0,-4.631,-30624000.0,-0.179,1.0,156798000.0,0.471,218140000.0,18040000.0,1.375958e+09,NaN,291248000.0,0.0,0.0,0.0,1.153225e+09,309866000.0,843359000.0,145697364.0,-12578000.0,0.0,0.0,23499000.0,0.0,23499000.0,-8679000.0,0.0,-23599000.0,-3799000.0,-37594000.0,-37594000.0,-37594000.0,0.0,0.0,-0.113,175882000.0,-19084000.0,149968000.0,0.000,-9.957,-0.455,-0.454,189673000.0,0.0,0.848,0.102,0.102,294747000.0,-635771000.0,333052000.0,333052000.0,0.0,NaN,NaN,NaN,NaN,483000.0,175179000.0,1.0,171731924.0,171343000.0,171343000.0,1.944,920452000.0,19514000.0,3728000.0,41297000.0,5.372,364413000.0
2015-05-15,ZPCM,ARQ,2015-03-31,2015-03-31,2019-06-12,0.0,6.942100e+05,NaN,694210.0,0.0,NaN,0.013,0.0,689080.0,689080.0,0.0,-68681.0,14.807,0.072,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.00,-68681.0,-68681.0,NaN,-68681.0,-68681.0,-68681.0,0.00,0.00,0.00,647327.0,NaN,647327.0,39314499.0,-233.0,-233

In [10]:
fundamentals_filtered_1st_filing.shape
#check shape of dataset


(1972, 110)

In [11]:
fundamentals_filtered_1st_filing.head()

,ticker,dimension,calendardate,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,assetsnc,assetturnover,bvps,capex,cashneq,cashnequsd,cor,consolinc,currentratio,de,debt,debtc,debtnc,debtusd,deferredrev,depamor,deposits,divyield,dps,ebit,ebitda,ebitdamargin,ebitdausd,ebitusd,ebt,eps,epsdil,epsusd,equity,equityavg,equityusd,ev,evebit,evebitda,fcf,fcfps,fxusd,gp,grossmargin,intangibles,intexp,invcap,invcapavg,inventory,investments,investmentsc,investmentsnc,liabilities,liabilitiesc,liabilitiesnc,marketcap,ncf,ncfbus,ncfcommon,ncfdebt,ncfdiv,ncff,ncfi,ncfinv,ncfo,ncfx,netinc,netinccmn,netinccmnusd,netincdis,netincnci,netmargin,opex,opinc,payables,payoutratio,pb,pe,pe1,ppnenet,prefdivis,price,ps,ps1,receivables,retearn,revenue,revenueusd,rnd,roa,roe,roic,ros,sbcomp,sgna,sharefactor,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital
datekey,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-05-12,ZYXI,ARQ,2015-03-31,2015-03-31,2020-10-27,0.0,6.416000e+06,NaN,5103000.0,1313000.0,NaN,-0.067,0.0,406000.0,406000.0,1245000.0,-904000.0,0.618,-4.117,4913000.0,4615000.0,298000.0,4913000.0,14000.0,0.0,0.0,0.714,0.0,-764000.0,-764000.0,-0.240,-764000.0,-764000.0,-896000.0,-0.03,-0.03,-0.03,-2082000.0,NaN,-2082000.0,11073959.0,-2.0,-2.356,163000.0,0.005,1.0,1938000.0,0.609,115000.0,132000.0,2.547000e+06,NaN,1640000.0,0.0,0.0,0.0,8.572000e+06,8261000.0,311000.0,6566959.0,285000.0,0.0,0.0,82000.0,0.0,82000.0,40000.0,0.0,163000.0,0.0,-896000.0,-896000.0,-896000.0,0.0,-8000.0,-0.281,2710000.0,-772000.0,2700000.0,0.0,-3.154,-1.159,-1.167,1140000.0,0.0,0.210,0.590,0.590,2924000.0,-7830000.0,3183000.0,3183000.0,0.0,NaN,NaN,NaN,NaN,0.0,2710000.0,1.0,31271234.0,31271234.0,31271234.0,0.102,6301000.0,0.0,0.0,0.0,0.201,-3158000.0
2015-05-12,ZSAN,ARQ,2015-03-31,2015-03-31,2020-11-13,0.0,6.642900e+07,NaN,56792000.0,9637000.0,NaN,101.222,-59000.0,55952000.0,55952000.0,0.0,-4583000.0,15.568,0.341,14680000.0,1457000.0,13223000.0,14680000.0,68000.0,633000.0,0.0,0.000,0.0,-4091000.0,-3458000.0,-15.862,-3458000.0,-4091000.0,-4583000.0,-9.40,-9.40,-9.40,49528000.0,NaN,49528000.0,72167833.0,NaN,NaN,-5486000.0,-11.212,1.0,218000.0,1.000,0.0,492000.0,2.150900e+07,NaN,0.0,0.0,0.0,0.0,1.690100e+07,3648000.0,13253000.0,113439833.0,54703000.0,0.0,61617000.0,-348000.0,0.0,60197000.0,-67000.0,-8000.0,-5427000.0,0.0,-4583000.0,-4583000.0,-4583000.0,0.0,0.0,-21.023,4369000.0,-4151000.0,1208000.0,0.0,2.290,NaN,NaN,9106000.0,0.0,190.000,NaN,NaN,142000.0,-143047000.0,218000.0,218000.0,3070000.0,NaN,NaN,NaN,NaN,29000.0,1299000.0,1.0,597052.0,489300.0,489300.0,0.446,66429000.0,0.0,0.0,0.0,135.763,53144000.0
2015-06-09,ZQKSQ,ARQ,2015-03-31,2015-04-30,2019-04-25,34517000.0,1.138592e+09,NaN,674279000.0,464313000.0,NaN,-0.085,-7025000.0,55395000.0,55395000.0,176254000.0,-37594000.0,2.176,-78.810,820767000.0,35285000.0,785482000.0,820767000.0,0.0,9707000.0,0.0,0.000,0.0,-15826000.0,-6119000.0,-0.018,-6119000.0,-15826000.0,-33866000.0,-0.22,-0.22,-0.22,-14633000.0,NaN,-14633000.0,911069364.0,-4.0,-4.631,-30624000.0,-0.179,1.0,156798000.0,0.471,218140000.0,18040000.0,1.375958e+09,NaN,291248000.0,0.0,0.0,0.0,1.153225e+09,309866000.0,843359000.0,145697364.0,-12578000.0,0.0,0.0,23499000.0,0.0,23499000.0,-8679000.0,0.0,-23599000.0,-3799000.0,-37594000.0,-37594000.0,-37594000.0,0.0,0.0,-0.113,175882000.0,-19084000.0,149968000.0,0.0,-9.957,-0.455,-0.454,189673000.0,0.0,0.848,0.102,0.102,294747000.0,-635771000.0,333052000.0,333052000.0,0.0,NaN,NaN,NaN,NaN,483000.0,175179000.0,1.0,171731924.0,171343000.0,171343000.0,1.944,920452000.0,19514000.0,3728000.0,41297000.0,5.372,364413000.0
2015-05-15,ZPCM,ARQ,2015-03-31,2015-03-31,2019-06-12,0.0,6.942100e+05,NaN,694210.0,0.0,NaN,0.013,0.0,689080.0,689080.0,0.0,-68681.0,14.807,0.072,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,-68681.0,-68681.0,NaN,-68681.0,-68681.0,-68681.0,0.00,0.00,0.00,647327.0,NaN,647327.0,39314499.0,-233.0,-233.315,-5983

In [12]:
len(fundamentals_filtered_1st_filing['ticker'].unique())

1951

### Tickers selected (not final)
- these are the tickers that we are going to look at from now on
- but we will have to continue modifying it if some of the other datasets do not have the same tickers

In [13]:
tickers = fundamentals_filtered_1st_filing['ticker'].unique()
# These are the tickers that we have selected to perform our strategies on

In [14]:
len(tickers)

1951

In [15]:
#we convert the tickers we want to a list, from now on these are the tickers we will use
type(tickers)


numpy.ndarray

In [16]:
tickers

array(['ZYXI', 'ZSAN', 'ZQKSQ', ..., 'AAPC', 'AAOI', 'AAME'], dtype=object)

- preliminarily, we start off with 1951 tickers, we will continue to trim down as we procceed

## Filtered Fundamentals
- we will filter the table by dimension of ART means as reported, and based on the tickers that we want. this will be used for judge the earnings dates

In [17]:
fundamentals_filtered  = table.loc[table['ticker'].isin(tickers)& (table['dimension'] == 'ARQ')]
#filter the table by dimension of ART means as reported, and based on the tickers that we want
#this will be used for judge the earnings dates

fundamentals_filtered.head()
# df.loc[df['channel'].isin(['sale','fullprice'])]


,ticker,dimension,calendardate,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,assetsnc,assetturnover,bvps,capex,cashneq,cashnequsd,cor,consolinc,currentratio,de,debt,debtc,debtnc,debtusd,deferredrev,depamor,deposits,divyield,dps,ebit,ebitda,ebitdamargin,ebitdausd,ebitusd,ebt,eps,epsdil,epsusd,equity,equityavg,equityusd,ev,evebit,evebitda,fcf,fcfps,fxusd,gp,grossmargin,intangibles,intexp,invcap,invcapavg,inventory,investments,investmentsc,investmentsnc,liabilities,liabilitiesc,liabilitiesnc,marketcap,ncf,ncfbus,ncfcommon,ncfdebt,ncfdiv,ncff,ncfi,ncfinv,ncfo,ncfx,netinc,netinccmn,netinccmnusd,netincdis,netincnci,netmargin,opex,opinc,payables,payoutratio,pb,pe,pe1,ppnenet,prefdivis,price,ps,ps1,receivables,retearn,revenue,revenueusd,rnd,roa,roe,roic,ros,sbcomp,sgna,sharefactor,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital
datekey,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-10-27,ZYXI,ARQ,2020-09-30,2020-09-30,2020-10-27,0.0,64965000.0,NaN,57553000.0,7412000.0,NaN,1.575,-105000.0,41511000.0,41511000.0,4296000.0,1333000.0,7.889,0.198,5177000.0,1729000.0,3448000.0,5177000.0,0.0,524000.0,0.0,0.000,0.0,1409000.0,1933000.0,0.097,1933000.0,1409000.0,1404000.0,0.04,0.04,0.04,54311000.0,NaN,54311000.0,459767337.0,39.0,33.981,-874000.0,-0.025,1.0,15730000.0,0.785,0.0,5000.0,21336000.0,NaN,5898000.0,0.0,0.0,0.0,10743000.0,7295000.0,3448000.0,496101337.0,24313000.0,0.0,25202000.0,-15000.0,0.0,25187000.0,-105000.0,0.0,-769000.0,0.0,1333000.0,1333000.0,1333000.0,0.0,0.0,0.067,14321000.0,1409000.0,2627000.0,0.0,9.134,48.476,46.065,6145000.0,0.0,14.28,7.223,7.170,9365000.0,21643000.0,20026000.0,20026000.0,0.0,NaN,NaN,NaN,NaN,730000.0,14321000.0,1.0,34740990.0,34486000.0,35476000.0,0.581,64965000.0,985000.0,71000.0,429000.0,1.884,50258000.0
2020-07-28,ZYXI,ARQ,2020-06-30,2020-06-30,2020-10-27,0.0,36759000.0,NaN,29833000.0,6926000.0,NaN,0.813,-357000.0,17198000.0,17198000.0,4061000.0,3017000.0,4.956,0.362,5531000.0,1749000.0,3782000.0,5531000.0,0.0,303000.0,0.0,0.000,0.0,4085000.0,4388000.0,0.228,4388000.0,4085000.0,4080000.0,0.09,0.09,0.09,27046000.0,NaN,27046000.0,630732842.0,49.0,44.758,2294000.0,0.069,1.0,15202000.0,0.789,0.0,5000.0,19072000.0,NaN,4707000.0,0.0,0.0,0.0,9802000.0,6020000.0,3782000.0,642399842.0,2331000.0,0.0,53000.0,-16000.0,0.0,37000.0,-357000.0,0.0,2651000.0,0.0,3017000.0,3017000.0,3017000.0,0.0,0.0,0.157,11117000.0,4085000.0,2201000.0,0.0,23.752,58.753,56.091,6241000.0,0.0,18.51,10.623,10.188,7264000.0,20310000.0,19263000.0,19263000.0,0.0,NaN,NaN,NaN,NaN,579000.0,11117000.0,1.0,34705556.0,33283000.0,34454000.0,0.579,36759000.0,545000.0,1063000.0,0.0,1.104,23813000.0
2020-04-28,ZYXI,ARQ,2020-03-31,2020-03-31,2020-10-27,0.0,33222000.0,NaN,25698000.0,7524000.0,NaN,0.711,-297000.0,14860000.0,14860000.0,3401000.0,2937000.0,4.405,0.424,5811000.0,1731000.0,4080000.0,5811000.0,0.0,242000.0,0.0,0.000,0.0,2458000.0,2700000.0,0.177,2700000.0,2458000.0,2454000.0,0.09,0.09,0.09,23397000.0,NaN,23397000.0,506762714.0,45.0,41.240,335000.0,0.010,1.0,11827000.0,0.777,0.0,4000.0,18339000.0,NaN,3429000.0,0.0,0.0,0.0,9914000.0,5834000.0,4080000.0,515811714.0,545000.0,0.0,221000.0,-11000.0,0.0,210000.0,-297000.0,0.0,632000.0,0.0,2937000.0,2937000.0,2937000.0,0.0,0.0,0.193,9369000.0,2458000.0,2292000.0,0.0,22.046,51.177,50.129,6264000.0,0.0,15.54,10.015,9.931,6549000.0,17293000.0,15228000.0,15228000.0,0.0,NaN,NaN,NaN,NaN,497000.0,9369000.0,1.0,33192517.0,32913000.0,34204000.0,0.463,33222000.0,985000.0,-483000.0,39000.0,1.009,19864000.0
2020-02-27,ZYXI,ARQ,2019-12-31,2019-12-31,2020-10-27,0.0,28277000.0,NaN,22566000.0,5711000.0,NaN,0.604,-28000.0,14369000.0,14369000.0,2821000.0,2947000.0,4.342,0.437,4683000.0,1256000.0,3427000.0,4683000.0,0.0,778000.0,0.0,0.000,0.0,3731000.0,4509000.0,0.318,4509000.0,3731000.0,3725000.0,0.09,0.09,0.09,19742000.0,NaN,19742000.0,381431037.0,32.0,29.977,2070000.0,0.063,1.0,11341000.0,0.8

In [19]:
fundamentals_filtered.shape

(35049, 110)

In [20]:
len(fundamentals_filtered['ticker'].unique())

1951

## Obtain other revelent dataset of filtered tickers 

<!-- ### Momentum
- Daily stock price and volume data (opening and closing price if possible) .
- Stock news and/or corporate actions that are 'ART' (actual reporting time)
- Valuation data (provided in SF1) .
- earnings beat -->



<!-- ### Technical Strategy
- instituional fund data to check for things like forced sale index inclusion
- short interest data for short squeeze strategy -->

## Event

In [21]:
event = pd.read_csv('../dataset/event.csv')
#read in the csv data for events

event_filtered = event[event['ticker'].isin(tickers)]
#filter out those tickers that we want

event_filtered.head()

,None,ticker,date,eventcodes
0,0,ABIO,2020-12-23,52|91
1,1,AGTC,2020-12-23,71|91
2,2,AIM,2020-12-23,11
3,3,AIRT,2020-12-23,11|23|71|91
4,4,ALSK,2020-12-23,35


In [22]:
df = event_filtered
df.head()

,None,ticker,date,eventcodes
0,0,ABIO,2020-12-23,52|91
1,1,AGTC,2020-12-23,71|91
2,2,AIM,2020-12-23,11
3,3,AIRT,2020-12-23,11|23|71|91
4,4,ALSK,2020-12-23,35


In [23]:
event_filtered = pd.concat([df,df.eventcodes.str.get_dummies(sep='|')],1)

event_filtered.head()

,None,ticker,date,eventcodes,11,12,13,14,21,22,23,24,25,26,31,32,33,34,35,37,41,42,51,52,53,54,55,56,57,58,71,81,91
0,0,ABIO,2020-12-23,52|91,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
1,1,AGTC,2020-12-23,71|91,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,2,AIM,2020-12-23,11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,AIRT,2020-12-23,11|23|71|91,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
4,4,ALSK,2020-12-23,35,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
event_filtered['ticker'].nunique()
# this is smaller than the number of tickers that we have previously

1948

In [24]:
event_filtered.drop(columns = ['None','eventcodes'], inplace = True)

In [25]:
event_filtered.shape

(132476, 31)

In [26]:
type(event_filtered['ticker'])

pandas.core.series.Series

In [28]:
event_filtered.head()

,ticker,date,11,12,13,14,21,22,23,24,25,26,31,32,33,34,35,37,41,42,51,52,53,54,55,56,57,58,71,81,91
0,ABIO,2020-12-23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
1,AGTC,2020-12-23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,AIM,2020-12-23,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,AIRT,2020-12-23,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
4,ALSK,2020-12-23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
len(event_filtered['ticker'])

132476

In [30]:
a = event_filtered['ticker'].unique()

In [31]:
type(tickers)

numpy.ndarray

In [32]:
print(np.setdiff1d(tickers, a))
# there are 3 tickers that are not in the events dataset, thus we will have to modify the tickers data to account for this

['ESI1' 'HDYN' 'HPCQ']


### Modified tickers
- there are 3 tickers that are not in the events dataset, thus we will have to modify the tickers data to account for this and come up with a new ticker csv

In [33]:
tickers = a

## Daily Stock Price and Volume Data
- this can be obtain from the Sharadar equity prices dataset
- it is an EOD price (end of day prices)

In [33]:
price_volume = pd.read_csv('../dataset/marketcap_price.csv')
#read in the csv file that contains daily stock price and volume

In [34]:
price_volume.head()

,ticker,date,open,high,low,close,volume,dividends,closeunadj,lastupdated
0,A,2020-12-09,116.380,116.960,114.480,116.400,1717581.0,0.0,116.400,2020-12-09
1,AA,2020-12-09,23.665,24.000,22.000,22.710,7018165.0,0.0,22.710,2020-12-09
2,AACG,2020-12-09,1.190,1.210,1.130,1.150,54904.0,0.0,1.150,2020-12-09
3,AACH,2020-12-09,0.021,0.025,0.017,0.025,135069.0,0.0,0.025,2020-12-09
4,AACQ,2020-12-09,10.400,10.500,10.270,10.300,1101151.0,0.0,10.300,2020-12-09


In [35]:
#filter price_volume according to the tickers we have chosen from before (tickers)
price_volume_filtered = price_volume[price_volume['ticker'].isin(tickers)]

In [36]:
# sam = price_volume.loc[price_volume['ticker'].isin(tickers)]
# # df.loc[df['favorite_color'].isin(array)]


In [37]:
len(tickers)

1948

In [38]:
sam.head(10)

NameError: name 'sam' is not defined

In [ ]:
price_volume_filtered.head(20)

In [ ]:
price_volume_filtered['ticker'].unique

In [ ]:
price_volume_filtered.tail()

In [ ]:
# 'AAOI' in price_volume_filtered['ticker']
#double check if it is correct
# i dk what's wrong with this

In [ ]:
price_volume_filtered.shape

In [ ]:
price_volume_filtered['ticker'].nunique()

In [ ]:
price_volume['ticker'].nunique()

In [ ]:
len(tickers)

In [ ]:
price_volume_filtered.head()
#see the first 5 rows

In [ ]:
price_volume_filtered.tail()


In [ ]:
price_volume_filtered.shape
#check shape

In [ ]:
price_volume_filtered.dtypes
#check datatypes of each row

In [ ]:
price_volume_sorted = price_volume_filtered.sort_values(['ticker','date'], ascending = [True, True])
#sort it by both the ticker symbol and date in ascending order

In [ ]:
price_volume_sorted.head(10)

In [ ]:
price_volume_sorted.isnull().sum()
# there is no null values, thus we do not have to clean it, but we still have to export it because we sorted it

In [ ]:
price_volume_sorted

In [ ]:
price_volume_sorted.dtypes
#check that the conversion is correct

In [ ]:
price_volume_sorted['ticker'].nunique()

## Daily Metrics of Filtered Tickers
- Filter out the daily financial metrics that contains our selected ticker

In [ ]:
daily_metrics = pd.read_csv('../dataset/daily_metrics.csv')

In [ ]:
daily_metrics = daily_metrics[daily_metrics['ticker'].isin(tickers)]

In [ ]:
#filter daily metrics by the date that is after 2015-03-31
daily_metrics= daily_metrics.loc[(daily_metrics['date'] >= '2015-03-31')]

In [ ]:
daily_metrics.head(10)
# EV, EV/EBIT, EV/EBITDA, EV/Rev are needed as well
# the marketcap and ev are in millions
# revenue stays the same every quarter

In [ ]:
daily_metrics = daily_metrics.drop(['Unnamed: 0'], axis =1)

In [ ]:
daily_metrics.dtypes
#check all the datatypes of daily_metrics_filtered

### Data Cleaning and Imputing of Data

In [ ]:
## check for null values
daily_metrics.isnull().sum()

In [ ]:
print(daily_metrics.isnull().values.sum())
#it seems that the data is clean

<!-- #### Dealing with NaN values
- Since the datatype of NA values are all float64, and from the data, it seems that they are all financial datas, thus they are likely to be related, or at least an extension of the data from the previous row. 
- Thus, we impute the Na values with the data from the previous row. -->

### Feature Engineering: Other Relevant Metrics
we need some more metrics to do our analysis, namely:
- revenue or sales
- ev/sales multiple

In [ ]:
sales = daily_metrics['ps'] * daily_metrics['marketcap']
sales.head()

In [ ]:
evsales = daily_metrics['ev'] / sales
evsales.head()

In [ ]:
daily_metrics['sales'] = sales
daily_metrics['evsales'] = evsales
# add both new metrics into the dataframe

In [ ]:
daily_metrics.head()

In [ ]:
daily_metrics_sorted = daily_metrics.sort_values(['ticker','date'], ascending = [True, True])


In [ ]:
daily_metrics_sorted.head()

In [ ]:
daily_metrics_sorted.tail()

# Final export to make sure all the things we exported are correct as we changed the tickers several times


### Export tickers

In [ ]:
len(tickers)

In [ ]:
tickers_df = pd.DataFrame(tickers)
tickers_df.to_csv('../dataset/tickers.csv', index = False, header = 'ticker')
#export the tickers we are concerned with

### Export fundamentals_filtered data

In [ ]:
fundamentals_filtered  = table.loc[table['ticker'].isin(tickers)& (table['dimension'] == 'ARQ')]
#filter the table by dimension of ART means as reported, and based on the tickers that we want
#this will be used for judge the earnings dates

fundamentals_filtered.head()
# df.loc[df['channel'].isin(['sale','fullprice'])]

fundamentals_filtered.to_csv('../dataset/fundamentals_filtered.csv', index = True)
#export it 


data = fundamentals_filtered
data_ticker = data['ticker'].unique()
data_ticker = data_ticker.tolist()
for i,value in enumerate(data_ticker):
    data[data['ticker'] == value].to_csv(f"../dataset/fundamentals_filtered_tickers/{value}.csv",index = True, na_rep = 'N/A')
# we export it in a way that each ticker has its own csv file with the datekey as index


In [ ]:
fundamentals_filtered.head()


In [ ]:
fundamentals_filtered['ticker'].nunique()

### Export event_filtered data

In [ ]:
event_filtered.to_csv('../dataset/event_filtered.csv',index = False)
#export

# split the csv file into multiple csv files based on the tickers
data = event_filtered

data_ticker = data['ticker'].unique()
data_ticker = data_ticker.tolist()

for i,value in enumerate(data_ticker):
    data[data['ticker'] == value].to_csv(f"../dataset/event_filtered_tickers/{value}.csv",index = False, na_rep = 'N/A')

In [ ]:
event_filtered.head()

In [ ]:
event_filtered['ticker'].nunique()

### Export price_volume_sorted data

In [ ]:
#export price_volume_sorted
price_volume_sorted.to_csv('../dataset/price_volume_sorted.csv', index=False)

# split the csv file into multiple csv files based on the tickers
data = price_volume_sorted

data_ticker = data['ticker'].unique()
data_ticker = data_ticker.tolist()

for i,value in enumerate(data_ticker):
    data[data['ticker'] == value].to_csv(f"../dataset/price_volume_tickers/{value}.csv", index = True, na_rep = 'N/A')

In [ ]:
price_volume_sorted.head()

In [ ]:
price_volume_sorted['ticker'].nunique()

### Export daily_metrics_sorted

In [ ]:
daily_metrics_sorted = daily_metrics.sort_values(['ticker','date'], ascending = [True, True])
#sort it by both the ticker symbol and date in ascending order, similar to that of price_volume_sorted

# Export the csv file
daily_metrics_sorted.to_csv('../dataset/daily_metrics_sorted.csv', index = False)

# split the csv file into multiple csv files based on the tickers
data = daily_metrics_sorted

data_ticker = data['ticker'].unique()
data_ticker = data_ticker.tolist()

for i,value in enumerate(data_ticker):
    data[data['ticker'] == value].to_csv(f"../dataset/daily_metrics_sorted_tickers/{value}.csv",index = False, na_rep = 'N/A')

In [ ]:
daily_metrics_sorted.head()

In [ ]:
daily_metrics_sorted['ticker'].nunique()

In [ ]:
below here don't touch, this is the final one. unless u want to add in more stuff

## Stock News and Corporate Actions that are 'ART' (Actual Reporting Time)


## Institutional and Insider Ownership 
- Filter the institutional and insider ownership based on the stock tickers and date that we want


In [ ]:
lets just focus on price volume for now, later then see if i need to split this into individual tickers as well

### Institutional Ownership

In [ ]:
insti_ownership = pd.read_csv('../dataset/SF3.csv')
# read in the csv file containing data of institutional ownership

In [ ]:
insti_ownership.head()
#check the first few rows

In [ ]:
insti_ownership.shape
#check the shape

In [ ]:
inst_ = insti_ownership.sort_values(['calendardate'], ascending = [True])
#sort institutional ownership according to date

In [ ]:
insti_ownership.head()



In [ ]:
insti_ownership.tail()

In [ ]:
insti_date = insti_ownership.loc[(insti_ownership['calendardate'] >= '2015-03-31')]
# we only want to filter out those after 2015-03-31

In [ ]:
insti_date.tail()

In [ ]:
unique_ticker = insti_date['ticker'].unique()
#find the unique elements in insti dataset

In [ ]:
unique_ticker = unique_ticker.tolist()
#convert it to list

In [ ]:
unique_ticker

In [ ]:
len(unique_ticker)
#check number of elements in unique_ticker

In [ ]:
# check_ticker = insti_date.ticker.unique().tolist()
# len(check_ticker)

In [ ]:
tickers

In [ ]:
type(tickers)

In [ ]:
tickers_original = tickers
#concert tickers to a list and name it tickers_original

In [ ]:
print(len(tickers_original))
print(len(unique_ticker))
#tickers_original is a subset of unique_ticker

In [ ]:
a_set = set(tickers_original) 
b_set = set(unique_ticker) 
#initiate the lists as sets

In [ ]:
common_insti = a_set.intersection(b_set)
#set a new variable as common element between the 2

In [ ]:
len(common_insti)
#find the number of common elements between the sets
#these are the elements we can work with for the testing of the institutional ownership strategy

In [ ]:
common_insti = list(common_insti)
#convert common_insti back to list

In [ ]:
type(common_insti)

#### Cleaning the Institutional Ownership Dataset
- filter out the insti_ownership dataset with common_insti 
- and then clean the new dataset of null values and such

In [ ]:
insti_ownership.head()

In [ ]:
insti_ownership_filtered = insti_ownership[insti_ownership['ticker'].isin(common_insti)]

# These are the tickers that we have selected to perform our strategies on for the institutional ownership strategy

In [ ]:
insti_ownership_filtered.head()

In [ ]:
insti_ownership_filtered.tail()

In [ ]:
insti_ownership_ticker_sorted = insti_ownership_filtered.sort_values(['ticker'], ascending = [True])
#sort by ticker symbol rather than date

In [ ]:
insti_ownership_ticker_sorted.tail()

#### Check for Null Values

In [ ]:
df1 = insti_ownership_ticker_sorted[insti_ownership_ticker_sorted.isna().any(axis=1)]
df1

In [ ]:
insti_ownership_ticker_sorted.dtypes

In [ ]:
insti_ownership_ticker_sorted.isnull().sum()

**looks like we have to dropna() since these data are independent of each other, and they belong to different stocks, thus imputing them does not make sense, we have to ignore these data altogether**

In [ ]:
insti_ownership_ticker_sorted = insti_ownership_ticker_sorted.dropna()

In [ ]:
insti_ownership_ticker_sorted.isnull().values.sum()

In [ ]:
insti_ownership_ticker_sorted.to_csv('../dataset/insti_ownership_ticker_sorted.csv')

### Insider Ownership

In [ ]:
leave this here first, i should start doing some momentum strategy, so this part is incomplete